# MLFlow Experiment 2: Vectorization Method

In [1]:
import mlflow

In [2]:
mlflow.set_tracking_uri("http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/")
mlflow.set_experiment("Exp 2 - BoW vs TfIdf")

2025/04/07 22:15:58 INFO mlflow.tracking.fluent: Experiment with name 'Exp 2 - BoW vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-experiments-bucket1/445903997996289143', creation_time=1744060557894, experiment_id='445903997996289143', last_update_time=1744060557894, lifecycle_stage='active', name='Exp 2 - BoW vs TfIdf', tags={}>

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
os.chdir("../")
os.getcwd()

'c:\\Users\\amman\\Documents\\MLOPS\\Youtube-Sentiment-Insights'

In [6]:
df = pd.read_csv("data/dataset.csv")
df.head()

,text,label
0,Anyone know movie is?,0
1,fact holding back equally aggressive :face_wit...,1
2,waiting next video be?,0
3,Thanks great video. understand DB continues ac...,0
4,Good person helping good people. America excep...,1


In [7]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [8]:
df.shape

(973405, 2)

In [11]:
# Create function to run experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    with mlflow.start_run():
        #set tags for the run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log parameters for the vectorizer
        mlflow.log_param("vectorizer_type", "CountVectorizer")
        mlflow.log_param("vectorizer_max_features", vectorizer.max_features)

        # Log parameters for the model
        n_estimators = 200
        max_depth = 15
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

In [12]:
# Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 10000 # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")

2025/04/07 22:30:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143/runs/e5a428fa5d4c4be6976cb198f636a6fe
🧪 View experiment at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143


2025/04/07 22:34:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143/runs/7d4a5d5eb3da47069dc53dafbf150e41
🧪 View experiment at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143


2025/04/07 22:38:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143/runs/9b4f535e67884c46a3f5d56c2aed212c
🧪 View experiment at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143


2025/04/07 22:47:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143/runs/bb66eaa73bfd443b9de2ac19473ac727
🧪 View experiment at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143


2025/04/07 22:52:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143/runs/af202fbe73b14b1b836a572395595d5e
🧪 View experiment at: http://ec2-18-209-5-28.compute-1.amazonaws.com:5000/#/experiments/445903997996289143
